In [21]:
import re
import os
import glob
import pandas as pd
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rc('pdf', fonttype=42)
import seaborn as sns
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict
from tqdm import tqdm
from ALLCools.plot import *

## perform motif enrichment

In [4]:
_dir = '500_result'
ct_list = ['shared', 'OPC_NN']
type_list = ['Hypo','Hyper']

In [5]:
groups = [f'{_ct}.{_type}' for _ct in ct_list for _type in type_list]

In [6]:
groups

['shared.Hypo', 'shared.Hyper', 'OPC_NN.Hypo', 'OPC_NN.Hyper']

In [11]:
#! mkdir motif_result

In [ ]:
findMotifsGenome.pl {bedfile} mm10 {outdir}/motif_{prefix} -size 200 &", file=motif_fh

In [20]:
with open(f'homer.sh', 'w') as file:
    for group in groups:
        _ct, _type = group.split('.')
        command = f"findMotifsGenome.pl {_dir}/{_type}.hotspot.{_ct}.bed mm10 motif_result/motif_{_type}.{_ct} -size 200 &"
        #command = f"annotatePeaks.pl {_dir}/{_type}.hotspot.{_ct}.bed mm10 > motif_result/{_type}.{_ct}.annotated.txt"
        file.write(command)
        file.write('\n')

## plot

In [17]:
_type = 'Hypo'
_ct = 'shared'

In [19]:
# result_df = pd.read_csv(f'motif_result/{_type}.{_ct}.annotated.txt')

In [23]:
p_value_cutoff = 0.01
min_fc = 1.2

In [22]:
known_motifs = glob.glob(f"motif_result/*/knownResults.txt")
len(known_motifs)

4

In [34]:
all_to_plot_df = []

for motif_result in tqdm(known_motifs):
    result_df = pd.read_csv(motif_result, sep="\t", index_col=0)
    result_df.index = [x.split("/")[0] for x in result_df.index]
    result_df['FC'] = result_df.iloc[:,4]/result_df.iloc[:,6]
    break
    result_df = result_df[(result_df['P-value'] < p_value_cutoff) & (result_df['FC'] > min_fc)]
    group = motif_result.split('/')[-2].split('motif_')[1]
    if result_df.shape[0] == 0:
        print(group)
    else:
        result_df['CellType']= group
        to_plot = result_df[['FC','Log P-value','CellType']].copy()
        to_plot = to_plot.sort_values('FC', ascending = False)

        all_to_plot_df.append(to_plot)

  0%|                                                     | 0/4 [00:00<?, ?it/s]
